# Test Modles to English Data

In [1]:
# Add option to read from padded or to rerun it again.
# =============================================================================
import numpy as np
from numpy import array
from numpy import argmax
import pandas as pd
import os,errno
from time import time
import keras
from keras.models import Sequential
from keras.preprocessing import sequence
from keras.layers import Dropout,LSTM, Lambda,Bidirectional,GRU,Dense
from keras.callbacks import ModelCheckpoint,TensorBoard#,TimeDistributed
#from keras.models import load_model
from sklearn.model_selection import train_test_split
from keras import backend as K
import sys
from preprocessing import restore
import tensorflow as tf
import random as rn

# =============================================================================


Using TensorFlow backend.


In [22]:
# =======================Program Parameters====================================
load_weights_flag = 1
# 0-> last wait | 1 max val_acc
last_or_max_val_acc = 0

Experiement_Name = 'Experiment_test'
layer_number = 3
#if u need one number for all layers add number alone
n_units = [200]
# 1->LSTM  , 2->GRU , 3->Bi-LSTM 
cell_mode = 2
drop_out_rate = 0.1
test_size_param=0.1
validation_split_param = 0.1
batch_size_param = 64
# 0 -> for test mode , 1 -> for train mode
learning_mode = 1

epochs_param = 5
#num of epoch should be wait when monitor don't change
earlystopping_patience=-1  

seed = 7


In [8]:
os.environ['PYTHONHASHSEED'] = '0'
np.random.seed(seed)
rn.seed(seed)


# =========================Functions ==========================================
class LossHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.losses = []

    def on_batch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
# =============================================================================

In [9]:
#===============================Concatinated Variables ========================
checkpoints_path ="../Experiement/checkpoints/"+Experiement_Name+"/"
check_points_file_path = checkpoints_path+ "/weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
board_log_dir="../Experiement/logs/"+Experiement_Name+"/"#+.format(time())

try:
    os.makedirs(board_log_dir)
    os.makedirs(checkpoints_path)
except OSError as e:
    if e.errno != errno.EEXIST:
        print("Can't create file for checkpoints or for logs please check ")
        raise

In [10]:
# =========================Data Loading========================================
X = restore("../data/data_matrix_X_binary_encoding.h5","X")
Y = restore("../data/data_matrix_Y_one_hot_encoding.h5","Y")
X =X[0:100]
Y =Y[0:100]
max_Bayt_length=X.shape[1]
char_dimension=X.shape[2]
numbber_of_bohor=Y.shape[1]
# =============================================================================


# ==============================Split Data=====================================
X_train, X_test, Y_train, Y_test=train_test_split(X, #bayts
                                                  Y, #classes
                                                  test_size=test_size_param, 
                                                  random_state=0)
# =============================================================================

In [11]:
X_train.shape

(90, 93, 5)

In [12]:
# =========================RNN models ================================
# create model
K.set_learning_phase(learning_mode) #set learning phase

model = Sequential()

# add layers
for n in range(layer_number):
    if len(n_units)==1 and layer_number>1:
        i=0
    else:
        if len(n_units)>=1 and len(n_units) != layer_number:
            sys.exit("pleas make length of n_units == layer_number or add only one element in n_units ")
        i=n
    # check if LSTM 
    if cell_mode==1:
        #check if first layer to add input_shape
        if n==0:
            # if NN as only one layer so shuld remove retun_sequences
            if layer_number == 1:
                model.add(LSTM(n_units[i],input_shape=(max_Bayt_length,char_dimension)))
            # if NN has many layers so should add return_sequences
            else:
                model.add(LSTM(n_units[i], return_sequences=True,input_shape=(max_Bayt_length,char_dimension)))
        # if it's not the first layer
        else:
            #check if last layer
            if layer_number-1 == n:
                model.add(LSTM(n_units[i]))
            else:
                model.add(LSTM(n_units[i], return_sequences=True))

    #check if GRU    
    elif cell_mode==2:

        #check if first layer to add input_shape
        if n==0:
            if layer_number  == 1:
            # if NN as only one layer so shuld remove retun_sequences
                model.add(GRU(n_units[i],input_shape=(max_Bayt_length,char_dimension)))
            # if NN has many layers so should add return_sequences
            else:
                model.add(GRU(n_units[i], return_sequences=True,input_shape=(max_Bayt_length,char_dimension)))
        # if it's not the first layer
        else:
            #check if last layer
            if layer_number-1 == n:
                model.add(GRU(n_units[i]))
            else:
                model.add(GRU(n_units[i], return_sequences=True))        
    #check if Bi-LSTM
    else:
        #check if first layer to add input_shape
        if n==0:
            # if NN as only one layer so shuld remove retun_sequences
            if layer_number  == 1:
                model.add(Bidirectional(LSTM(n_units[i]),
                                        input_shape=(max_Bayt_length, char_dimension)))
            # if NN has many layers so should add return_sequences
            else:
                model.add(Bidirectional(LSTM(n_units[i], return_sequences=True),
                                        input_shape=(max_Bayt_length, char_dimension)))
        # if it's not the first layer
        else:
            #check if last layer
            if layer_number-1 == n:
                model.add(Bidirectional(LSTM(n_units[i])))
            else:
                model.add(Bidirectional(LSTM(n_units[i], return_sequences=True)))
    #check if there Dopout or not
    if drop_out_rate != 0:
        model.add(Dropout(drop_out_rate,seed=seed))

                      
#add softmax layer
model.add(Dense(units = numbber_of_bohor,activation = 'softmax'))

In [23]:
#==================================check to load last epoch====================
# load weights
if(load_weights_flag == 1):
    try:
        print("loading last weights in last epoch")
        #List all avialble checkpoints into the directory
        checkpoints_path_list = os.listdir(checkpoints_path)
        all_checkpoints_list = [os.path.join(checkpoints_path,i) for i in checkpoints_path_list]
        #Get the last inserted weight into the checkpoint_path
        all_checkpoints_list_sorted = sorted(all_checkpoints_list, key=os.path.getmtime)
        if(last_or_max_val_acc == 0):
            print ("last check point")
            if "last" in all_checkpoints_list_sorted[-1]:
                print(all_checkpoints_list_sorted[-1])
                max_weight_checkpoints =  all_checkpoints_list_sorted[-1]
                #load weights
#                 model = keras.models.load_model(max_weight_checkpoints)
            else:
                sys.exit(0)
        else:
            print ("max_weight_checkpoints")
            print(all_checkpoints_list_sorted[-2])
            max_weight_checkpoints =  all_checkpoints_list_sorted[-2]
#             #load weights
#             model = keras.models.load_model(max_weight_checkpoints)

    except IOError:
        print('An error occured trying to read the file.')
    except:
        if "last" not in  all_checkpoints_list_sorted[-1]:
            sys.exit("Last epoch don't exist in this modle , you can make last_or_max_val_acc=1 to load the epoch has max val_acc")
        else:
            print("No wieghts avialable \n check the paths")


loading last weights in last epoch
last check point
../Experiement/checkpoints/Experiment_test/weights-improvement-last-epoch.hdf5


In [24]:
#==================================compile model===============================        
# Compiling the RNN
model.compile(optimizer = 'adam', 
              loss='categorical_crossentropy',
              metrics = ['accuracy'])

In [25]:
class last_epoch_saver(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        #save last epoch weghits 
        self.model.save(checkpoints_path+"weights-improvement-last-epoch.hdf5")
        print("Save last epoch Done! ....")


#==============================Callbacks========================================
checkpoint = ModelCheckpoint(check_points_file_path, 
                             monitor='val_acc', 
                             verbose=1,
                             save_best_only=True, 
                             mode='max')

tensorboard  = keras.callbacks.TensorBoard(log_dir=board_log_dir , 
                                           histogram_freq=0, 
                                           batch_size=50, 
                                           write_graph=True, 
                                           write_grads=True, 
                                           write_images=True, 
                                           embeddings_freq=0, 
                                           embeddings_layer_names=None, 
                                           embeddings_metadata=None)

earlystopping = keras.callbacks.EarlyStopping(monitor='val_acc',
                                             min_delta=0,
                                             patience=earlystopping_patience,
                                             verbose=1,
                                             mode='auto')
last_epoch_saver_ = last_epoch_saver()


if earlystopping_patience ==-1:
    callbacks_list = [checkpoint,tensorboard,last_epoch_saver_]
    print("Add  checkpoint - tensorboard - last_epoch_saver_")
else:
    callbacks_list = [checkpoint,tensorboard,earlystopping,last_epoch_saver_]
    print("Add  checkpoint - tensorboard - earlystopping - last_epoch_saver_")

    

callbacks_list = [checkpoint,my]
    
print(model.summary())
#==============================================================================


Add  checkpoint - tensorboard
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_1 (GRU)                  (None, 93, 200)           123600    
_________________________________________________________________
dropout_1 (Dropout)          (None, 93, 200)           0         
_________________________________________________________________
gru_2 (GRU)                  (None, 93, 200)           240600    
_________________________________________________________________
dropout_2 (Dropout)          (None, 93, 200)           0         
_________________________________________________________________
gru_3 (GRU)                  (None, 200)               240600    
_________________________________________________________________
dropout_3 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)        

In [26]:
#=============================Fitting Model====================================
# Fitting the RNN to the Training set
hist = model.fit(X_train, 
                 Y_train, 
                 validation_split = validation_split_param, 
                 epochs=epochs_param, 
                 batch_size=batch_size_param, 
                 callbacks=callbacks_list,
                 verbose=1)


Train on 81 samples, validate on 9 samples
Epoch 1/5
64/81 [======================>.......] - ETA: 1s - loss: 1.4156e-07 - acc: 1.0000Epoch 00000: val_acc improved from -inf to 1.00000, saving model to ../Experiement/checkpoints/Experiment_test//weights-improvement-00-1.00.hdf5
Save last epoch Done! ....
81/81 [==============================] - 7s - loss: 1.4349e-07 - acc: 1.0000 - val_loss: 1.1921e-07 - val_acc: 1.0000
Epoch 2/5
64/81 [======================>.......] - ETA: 0s - loss: 1.2945e-07 - acc: 1.0000Epoch 00001: val_acc did not improve
Save last epoch Done! ....
81/81 [==============================] - 4s - loss: 1.2951e-07 - acc: 1.0000 - val_loss: 1.1921e-07 - val_acc: 1.0000
Epoch 3/5
64/81 [======================>.......] - ETA: 0s - loss: 1.3504e-07 - acc: 1.0000Epoch 00002: val_acc did not improve
Save last epoch Done! ....
81/81 [==============================] - 4s - loss: 1.3245e-07 - acc: 1.0000 - val_loss: 1.2583e-07 - val_acc: 1.0000
Epoch 4/5
64/81 [=============

In [ ]:
#save last epoch weghits 
model.save(checkpoints_path+"weights-improvement-last-epoch.hdf5")
print("Save last epoch Done! ....")

In [241]:
#===========================Evaluate model=====================================
# Final evaluation of the model
scores = model.evaluate(X_test, Y_test, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

711/711 [==============================] - 13s    
Accuracy: 64.14%


'../Experiement/checkpoints/Experiment_test//weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5'

# Clean English Data

In [101]:
import math
import string
import re

In [160]:
dataset = pd.read_csv('../data/english_dataset.csv', encoding = "utf-8",index_col=0)

In [78]:
len(dataset)

7105

In [77]:
dataset.columns = ['Verse','Meter']

In [78]:
dataset.columns

Index(['Verse', 'Meter'], dtype='object')

In [79]:
dataset['Verse'] = dataset['Verse'].map(lambda x: x.lower())
dataset['Meter'] = dataset['Meter'].map(lambda x: x.lower())

In [80]:
dataset.head(1)

,Verse,Meter
0,humpty dumpty sat on a wall,dactyl


In [81]:
english_alphapets=list(string.ascii_lowercase)+[" ","\'"]
our_alphabets = "".join(english_alphapets)

In [82]:
dataset['Verse']=dataset['Verse'].apply(lambda x: re.sub(r'[^'+our_alphabets+']','',str(x))).apply(
                                          lambda x: re.sub(r'  *'," ",x))

In [83]:
alphabets = set() 
def f(x): 
    global alphabets
    alphabets|=set(x)
total = math.ceil(len(dataset)/100)
for i in range(total):
    s=i*100
    e=(i*100)+100
    if e>len(dataset):
        e=len(dataset)
    new = dataset[s:e]['Verse'].map(f)

len(alphabets)

28

In [84]:
alphabets

{' ',
 "'",
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z'}

In [85]:
dataset['Meter']=dataset['Meter'].apply(lambda x: re.sub(r'[^'+our_alphabets+']','',str(x))).apply(
                                          lambda x: re.sub(r'  *'," ",x))

In [89]:
alphabets = set() 
def f(x): 
    global alphabets
    alphabets|=set(x)
total = math.ceil(len(dataset)/100)
for i in range(total):
    s=i*100
    e=(i*100)+100
    if e>len(dataset):
        e=len(dataset)
    new = dataset[s:e]['Meter'].map(f)

len(alphabets)

16

In [90]:
dataset['Verse'].map(len).max()

93

In [91]:
dataset['Verse']=dataset['Verse'].apply(lambda x: x.strip())

In [92]:
dataset['Verse'].map(len).max()

93

In [94]:
dataset['Meter'].unique()

array(['dactyl', 'pyrrhic', 'spondee', 'trochee', 'iambic', 'anapaest'], dtype=object)

In [95]:
dataset.to_csv('../data/english_dataset.csv', encoding = "utf-8")

In [98]:
dataset=pd.read_csv('../data/english_dataset.csv',index_col=0)

,Verse,Meter
0,humpty dumpty sat on a wall,dactyl
1,humpty dumpty had a great fall,dactyl
2,humptys mother was very sad,dactyl
3,to see her son in the hospital,dactyl
4,humpty dumpty was slowly dying,dactyl
5,humpty dumpty was painfully frying,dactyl
6,he said that there was only one cure,dactyl
7,just give me one popsicle,dactyl
8,adam and eve,dactyl
9,john milton,dactyl


In [91]:
dataset['Meter'].value_counts()

iambic      2155
dactyl      1993
anapaest    1022
trochee     1000
pyrrhic      740
spondee      195
Name: Meter, dtype: int64


# make one-hot encoding

In [249]:
import string
import pandas as pd
import numpy as np 

In [250]:
dataset = pd.read_csv('../data/english_dataset.csv', encoding = "utf-8",index_col=0)

In [251]:
dataset.head(1)

,Verse,Meter
0,humpty dumpty sat on a wall,dactyl


In [252]:
maxlen= dataset['Verse'].map(len).max()
maxlen

93

In [253]:
alphabet1=list(string.ascii_lowercase)+[' ', '\''] #Add space and apostrophe to our alphabets
alphabet = dict((c, i) for i, c in enumerate(alphabet1))
alphabet_ = dict((i, c) for i, c in enumerate(alphabet1))

In [254]:
def one(num):
    a = np.zeros(len(alphabet1))
    a[num] = 1
    return a

def one_hot(verse):
    d = maxlen - len(verse)
    verse = verse + " "*d
    data = [one(alphabet[char]) for char in verse]
#     return np.asarray(tf.Session().run(tf.one_hot(indices=data,depth=len(alphabet1))).tolist())
    return np.asarray(data)

In [255]:
res = dataset['Verse'].map(one_hot)

In [256]:
result= np.stack(res,axis=0)

In [257]:
result.shape

(7105, 93, 28)

In [259]:
dataset.head(2)

,Verse,Meter
0,humpty dumpty sat on a wall,dactyl
1,humpty dumpty had a great fall,dactyl


In [260]:
def from_one_hot_to_char(arr): 
    a = []
    for ii in arr:
        a.append([alphabet_[i] for i,l in enumerate(ii) if int(l)==1][0])
    return "".join(a)

In [261]:
import h5py
def save(nameOfFile,nameOfDataset,dataVar):
    h5f = h5py.File(nameOfFile, 'w')
    h5f.create_dataset(nameOfDataset, data=dataVar)
    h5f.close()
#---------------------------Retrive Encoding--------------------------------------
def restore (nameOfFile,nameOfDataset):
    h5f = h5py.File(nameOfFile,'r')
    matrix = h5f[nameOfDataset][:]
    h5f.close()
    return matrix


In [262]:
save("../data/data_matrix_X_one_hot_encoding.h5","X",result)